## Enhanced Plastic Degradation Using Optimized PETase Peptides for Dual PETase-PLAase Activity


### step1: find K

##### import

In [74]:
!pip install biopython


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\xuech\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [75]:
from Bio import Align

##### read files

In [76]:
# read file D
with open('SetD_84seq_nucleo_genomes.txt', 'r') as file:
    lines = file.readlines()

# Save each sequence in a list and discard headers
D = []
current_seq = []
for line in lines:
    if line.startswith('>'):
        if current_seq:
            D.append(''.join(current_seq))
            current_seq = []
    else:
        current_seq.append(line.strip())
if current_seq:
    D.append(''.join(current_seq))

print(D[0][:10])  # Print the first 10 characters of the first sequence to verify
print(D[1][:10])  # Print the first 10 characters of the second sequence to verify

ACCGAGAGGT
TTCCAGCAGG


##### save found M1 and S1

In [100]:
S1 = "ATGAACTTTCCCCGCGCTTCCCGCCTGATGCAGGCCGCCGTTCTCGGCGGGCTGATGGCCGTGTCGGCCGCCGCCACCGCCCAGACCAACCCCTACGCCCGCGGCCCGAACCCGACAGCCGCCTCACTCGAAGCCAGCGCCGGCCCGTTCACCGTGCGCTCGTTCACCGTGAGCCGCCCGAGCGGCTACGGCGCCGGCACCGTGTACTACCCCACCAACGCCGGCGGCACCGTGGGCGCCATCGCCATCGTGCCGGGCTACACCGCGCGCCAGTCGAGCATCAAATGGTGGGGCCCGCGCCTGGCCTCGCACGGCTTCGTGGTCATCACCATCGACACCAACTCCACGCTCGACCAGCCGTCCAGCCGCTCGTCGCAGCAGATGGCCGCGCTGCGCCAGGTGGCCTCGCTCAACGGCACCAGCAGCAGCCCGATCTACGGCAAGGTCGACACCGCCCGCATGGGCGTGATGGGCTGGTCGATGGGCGGTGGCGGCTCGCTGATCTCGGCGGCCAACAACCCGTCGCTGAAAGCCGCGGCGCCGCAGGCCCCGTGGGACAGCTCGACCAACTTCTCGTCGGTCACCGTGCCCACGCTGATCTTCGCCTGCGAGAACGACAGCATCGCCCCGGTCAACTCGTCCGCCCTGCCGATCTACGACAGCATGTCGCGCAATGCGAAGCAGTTCCTCGAGATCAACGGTGGCTCGCACTCCTGCGCCAACAGCGGCAACAGCAACCAGGCGCTGATCGGCAAGAAGGGCGTGGCCTGGATGAAGCGCTTCATGGACAACGACACGCGCTACTCCACCTTCGCCTGCGAGAACCCGAACAGCACCCGCGTGTCGGACTTCCGCACCGCGAACTGCAGC"

In [ ]:
M1 = "GACAGCATCGCCCCGGTCAACTCGTCCGCCCTGCCGATCTACGACAGCATGTCGCGCAATGCGAAGCAGTTCCTCGAGATCAACGGTGGCTCGCAC"

##### define homologous function for comparing two strings

In [116]:
aligner = Align.PairwiseAligner()

def is_homologous(seq1, seq2, sim=80):
    #global alignment
    aligner.mode = 'local'
    alignments = [aln for aln in aligner.align(seq1, seq2) if aln.shape[1] > 2] #eliminate alginments with less than 3 matches 
    best_alignment = alignments[0] if alignments else None
    identity = best_alignment.score / min(len(seq1), len(seq2)) * 100 if best_alignment else 0 #alignment score is the number of matches divided by min length of two strings
    return identity>= sim, best_alignment[0] if alignments else None


In [ ]:
# test case should return false
seq1 = "ATTTGGGAAA"
seq2 = "TGATGTATAG"
similarity_threshold = 80

print(is_homologous(seq1, seq2, similarity_threshold))

(False, 'AT-T-T-G')


In [114]:
def is_homologous_to_pair(T, S, M, sim=80):
    # Check if T is homologous to S
    #return if a match is found and the best local alignment found
    is_homologous_to_S, alignment_S = is_homologous(T, S, sim)
    if is_homologous_to_S:
        aligned_seq_S = str(alignment_S)
        is_homologous_to_M, alignment_M = is_homologous(aligned_seq_S, M, sim)
        if is_homologous_to_M:
            return True, (alignment_S, alignment_M) 

    return False, (None, None)

In [117]:
#test case should return
seq1 = "ATTTGGGAAA"
seq2 = "TGGG"
M = "GG"
similarity_threshold = 80

print(is_homologous_to_pair(seq1, seq2, M, similarity_threshold))

(True, ('TTTGGG', 'GGG'))


In [ ]:
# def find_K_parellel(D, S, M, sim=50):
#     K = []
#     len_S = len(S)
#     import concurrent.futures

#     def process_sequence(seq):
#         alignments_pair = [] #return the alignment pair of D "both substing of D"
#         is_homologous, alignments = is_homologous_to_pair(seq, S, M, sim)
#         if is_homologous:
#             alignments_pair.append(alignments)
#         return alignments_pair

#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         results = list(executor.map(process_sequence, D))

#     # Flatten the list of lists
#     K = [item for sublist in results for item in sublist]
#     print(K)
#     return K

In [ ]:
def find_K(D, S, M, sim=80):
    K = []
    for seq in D:
        is_homologous, alignments = is_homologous_to_pair(seq, S, M, sim)
        if is_homologous:
            K.append(alignments)
    return K

In [ ]:
# # read file D
# with open('D1.txt', 'r') as file:
#     lines = file.readlines()
# D1 = ''.join([line.strip() for line in lines[1:]])
# #print(D[:100])

In [110]:
K_set = find_K(D[:2], S1, M1, 80) #testing first two bacteria sequence 
print(K_set)

KeyboardInterrupt: 